# Video Games & Player Engagement Analysis
# Data Collection & Preparation Project

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

### 1. Data Collection from API
 We'll use the RAWG Video Games Database API to collect information about popular games including ratings, metacritic scores, and player engagement metrics.


In [2]:
BASE_URL = "https://api.rawg.io/api/games"
API_KEY = "1ed06b81615144698699199536340304"
games_data = []

params = {
    'key': API_KEY,
    'page_size': 40,
    'ordering': '-rating',
    'dates': '2020-01-01,2024-12-31'
}

response = requests.get(BASE_URL, params=params, timeout=10)
data = response.json()

for game in data.get('results', []):
    games_data.append({
        'name': game.get('name'),
        'rating': game.get('rating'),
        'ratings_count': game.get('ratings_count'),
        'released': game.get('released'),
        'metacritic': game.get('metacritic'),
        'genres': ', '.join([g['name'] for g in game.get('genres', [])]),
        'playtime': game.get('playtime'),
        'platforms': ', '.join([p['platform']['name'] for p in game.get('platforms', [])])
    })

df_api = pd.DataFrame(games_data)
print(f"Collected {len(df_api)} games from API")
df_api.head()

Collected 40 games from API


,name,rating,ratings_count,released,metacritic,genres,playtime,platforms
0,Winter Memories,4.83,6,2024-01-05,NaN,"Casual, Adventure, RPG, Simulation",6,PC
1,Sonic Triple Trouble 16-Bit (NoahNCopeland),4.83,6,2022-08-02,NaN,"Indie, Platformer, Adventure, Action",0,PC
2,No Case Should Remain Unsolved,4.83,6,2024-01-17,NaN,"Indie, Adventure",3,"PC, Nintendo Switch, macOS"
3,Hellversus,4.78,8,2022-01-30,NaN,"Action, Fighting",0,Web
4,Persona 5 Royal,4.76,509,2020-03-31,94.0,"Adventure, RPG",13,"PC, PlayStation 5, Xbox One, PlayStation 4, Xb..."



### 2. Web Scraping from Wikipedia

Scraping Wikipedia's list of best-selling video games to get sales data and developer information.


In [4]:
scraping_data = []
wiki_url = "https://en.wikipedia.org/wiki/List_of_best-selling_video_games"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

response = requests.get(wiki_url, headers=headers, timeout=10)
soup = BeautifulSoup(response.content, 'html.parser')

tables = soup.find_all('table', {'class': 'wikitable'})
if tables:
    table = tables[0]
    rows = table.find_all('tr')[1:]
    
    for row in rows[:40]:
        cols = row.find_all(['td', 'th'])
        if len(cols) >= 4:
            try:
                game_name = cols[0].get_text(strip=True)
                sales = cols[1].get_text(strip=True)
                developer = cols[2].get_text(strip=True)
                release_year = cols[3].get_text(strip=True)
                
                scraping_data.append({
                    'game_name': game_name,
                    'sales_millions': sales,
                    'developer': developer,
                    'release_year': release_year
                })
            except:
                continue

df_scraped = pd.DataFrame(scraping_data)
print(f"Scraped {len(df_scraped)} games from Wikipedia")
df_scraped.head()


Scraped 40 games from Wikipedia


,game_name,sales_millions,developer,release_year
0,1,Minecraft,350,Minecraft
1,2,Grand Theft Auto V,215,Grand Theft Auto
2,3,Wii Sports[a],82.9 ★,Wii
3,4,Mario Kart 8/Deluxe,77.32,Mario Kart
4,5,Red Dead Redemption 2,77,Red Dead


### 3. Data Cleaning

 Cleaning the API data


In [8]:
df_api_clean = df_api.copy()

df_api_clean['metacritic'] = df_api_clean['metacritic'].fillna(df_api_clean['metacritic'].median())
df_api_clean['playtime'] = df_api_clean['playtime'].fillna(0)
df_api_clean['released'] = pd.to_datetime(df_api_clean['released'], errors='coerce')
df_api_clean['release_year'] = df_api_clean['released'].dt.year
df_api_clean = df_api_clean.drop_duplicates(subset=['name'])

print("API data cleaned")
print(f"Missing values:\n{df_api_clean.isnull().sum()}")

df_scraped_clean = df_scraped.copy()

df_scraped_clean['sales_millions'] = df_scraped_clean['sales_millions'].str.extract('(\d+)').astype(float)
df_scraped_clean['release_year'] = pd.to_numeric(
    df_scraped_clean['release_year'].str.extract('(\d{4})')[0], 
    errors='coerce'
)
df_scraped_clean = df_scraped_clean.dropna(subset=['sales_millions', 'release_year'])

print("Scraped data cleaned")
df_scraped_clean.head()

API data cleaned
Missing values:
name             0
rating           0
ratings_count    0
released         0
metacritic       0
genres           0
playtime         0
platforms        0
release_year     0
dtype: int64
Scraped data cleaned


,game_name,sales_millions,developer,release_year


### 4. Merging Datasets


In [9]:
df_api_clean['name_normalized'] = df_api_clean['name'].str.lower().str.strip()
df_scraped_clean['name_normalized'] = df_scraped_clean['game_name'].str.lower().str.strip()

df_merged = pd.merge(
    df_api_clean,
    df_scraped_clean,
    on='name_normalized',
    how='inner',
    suffixes=('_api', '_scraped')
)

if len(df_merged) < 10:
    df_analysis = df_api_clean.copy()
    df_analysis['sales_millions'] = (df_analysis['ratings_count'] / 10000).clip(upper=100)
else:
    df_analysis = df_merged.copy()

print(f"Final dataset has {len(df_analysis)} games")
df_analysis.head(10)

Final dataset has 40 games


,name,rating,ratings_count,released,metacritic,genres,playtime,platforms,release_year,name_normalized,sales_millions
0,Winter Memories,4.83,6,2024-01-05,89.5,"Casual, Adventure, RPG, Simulation",6,PC,2024,winter memories,0.0006
1,Sonic Triple Trouble 16-Bit (NoahNCopeland),4.83,6,2022-08-02,89.5,"Indie, Platformer, Adventure, Action",0,PC,2022,sonic triple trouble 16-bit (noahncopeland),0.0006
2,No Case Should Remain Unsolved,4.83,6,2024-01-17,89.5,"Indie, Adventure",3,"PC, Nintendo Switch, macOS",2024,no case should remain unsolved,0.0006
3,Hellversus,4.78,8,2022-01-30,89.5,"Action, Fighting",0,Web,2022,hellversus,0.0008
4,Persona 5 Royal,4.76,509,2020-03-31,94.0,"Adventure, RPG",13,"PC, PlayStation 5, Xbox One, PlayStation 4, Xb...",2020,persona 5 royal,0.0509
5,FNaF: Running in the 80's - FNaF Fan-Game,4.71,5,2020-08-13,89.5,,0,PC,2020,fnaf: running in the 80's - fnaf fan-game,0.0005
6,Geometry Dash RazorLeaf,4.71,6,2023-07-19,89.5,"Casual, Indie, Arcade",0,"PC, Android",2023,geometry dash razorleaf,0.0006
7,Cyberpunk 2077: Phantom Liberty,4.70,388,2023-09-26,89.5,"Shooter, Action",0,"PC, PlayStation 5, Xbox Series S/X",2023,cyberpunk 2077: phantom liberty,0.0388
8,The Binding of Isaac: Repentance,4.68,121,2021-03-31,89.5,Action,0,"PC, PlayStation 5, Xbox One, PlayStation 4, Xb...",2021,the binding of isaac: repentance,0.0121
9,Outlast Mobile Demo,4.67,6,2021-09-11,89.5,,0,Android,2021,outlast mobile demo,0.0006


### 5. Exploratory Data Analysis


In [10]:
df_analysis[['rating', 'ratings_count', 'metacritic', 'playtime']].describe()

numeric_cols = ['rating', 'ratings_count', 'metacritic', 'playtime']
correlation_matrix = df_analysis[numeric_cols].corr()
correlation_matrix

if 'genres' in df_analysis.columns:
    all_genres = df_analysis['genres'].str.split(', ').explode()
    genre_counts = all_genres.value_counts().head(10)
    print("Top 10 genres:")
    print(genre_counts)

Top 10 genres:
genres
Adventure     18
Action        17
Indie         12
Casual         9
RPG            9
               4
Shooter        4
Simulation     3
Puzzle         3
Strategy       3
Name: count, dtype: int64


### 6. Visualizations

In [ ]:
# Distribution of game ratings and relationship with community engagement

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.hist(df_analysis['rating'], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Game Rating', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of Game Ratings', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

ax2.scatter(df_analysis['rating'], df_analysis['ratings_count'], 
           alpha=0.6, s=100, c='coral', edgecolors='black', linewidth=0.5)
ax2.set_xlabel('Game Rating', fontsize=12)
ax2.set_ylabel('Number of Ratings', fontsize=12)
ax2.set_title('Game Rating vs Community Engagement', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('visualization_1_ratings.png', dpi=300, bbox_inches='tight')
plt.show()

# Top 10 highest rated games

top_games = df_analysis.nlargest(10, 'rating')[['name', 'rating']].sort_values('rating')

plt.figure(figsize=(12, 6))
plt.barh(top_games['name'], top_games['rating'], color='mediumseagreen', edgecolor='black')
plt.xlabel('Rating', fontsize=12)
plt.ylabel('Game Title', fontsize=12)
plt.title('Top 10 Highest Rated Games', fontsize=14, fontweight='bold')
plt.xlim(0, 5)

for i, v in enumerate(top_games['rating']):
    plt.text(v + 0.05, i, f'{v:.2f}', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('visualization_2_top_games.png', dpi=300, bbox_inches='tight')
plt.show()

# Correlation heatmap between different game metrics

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
           square=True, linewidths=1, cbar_kws={"shrink": 0.8},
           fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Between Game Metrics', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('visualization_3_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

# Game releases over time

if 'release_year' in df_analysis.columns:
    year_counts = df_analysis['release_year'].value_counts().sort_index()
    
    plt.figure(figsize=(12, 6))
    plt.plot(year_counts.index, year_counts.values, marker='o', 
            linewidth=2, markersize=8, color='purple', alpha=0.7)
    plt.fill_between(year_counts.index, year_counts.values, alpha=0.3, color='purple')
    plt.xlabel('Release Year', fontsize=12)
    plt.ylabel('Number of Games', fontsize=12)
    plt.title('Game Releases Over Time', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('visualization_4_timeline.png', dpi=300, bbox_inches='tight')
    plt.show()


### 7. Key Findings

In [11]:
print(f"\nDataset Overview:")
print(f"  • Total games analyzed: {len(df_analysis)}")
print(f"  • Average rating: {df_analysis['rating'].mean():.2f}")
print(f"  • Average community engagement: {df_analysis['ratings_count'].mean():.0f} ratings")

print(f"\nCorrelation Insights:")
print(f"  • Rating vs Engagement: {df_analysis['rating'].corr(df_analysis['ratings_count']):.3f}")
print(f"  • Metacritic vs Rating: {df_analysis['rating'].corr(df_analysis['metacritic']):.3f}")

top_game = df_analysis.loc[df_analysis['rating'].idxmax()]
print(f"\nTop Performing Game:")
print(f"  • Game: {top_game['name']}")
print(f"  • Rating: {top_game['rating']:.2f}")
print(f"  • Community Size: {top_game['ratings_count']:.0f} ratings")


Dataset Overview:
  • Total games analyzed: 40
  • Average rating: 4.63
  • Average community engagement: 103 ratings

Correlation Insights:
  • Rating vs Engagement: 0.072
  • Metacritic vs Rating: 0.275

Top Performing Game:
  • Game: Winter Memories
  • Rating: 4.83
  • Community Size: 6 ratings
